In [1]:
using Pkg; Pkg.activate("../../../FermiCG/")
using FermiCG
using PyCall
using Plots
using LinearAlgebra
using Printf

  Activating environment at `~/code/FermiCG/Project.toml`


In [12]:
molecule = "
He         0.83846       -0.83846        0.83846
He         0.83846       -0.83846       -0.83846
He         0.83846        0.83846       -0.83846
He         0.83846        0.83846        0.83846
He        -0.83846        0.83846        0.83846
He        -0.83846        0.83846       -0.83846
He        -0.83846       -0.83846       -0.83846
He        -0.83846       -0.83846        0.83846
"
atoms = []
for (li,line) in enumerate(split(rstrip(lstrip(molecule)), "\n"))
    l = split(line)
    push!(atoms, Atom(li, l[1], parse.(Float64,l[2:4])))
end

#basis = "augccpvdz" #9 orbs on each He
basis = "cc-pvdz" #5 orbs on each He
#basis = "cc-pvtz" # 6 orbs on each He
#basis = "sto-3g"
#basis = "631g**" #5 each atom
#basis = "631g*" #2 orbs each atom

# Create FermiCG.Molecule type
mol = Molecule(0,1,atoms,basis)

pyscf = pyimport("pyscf")
pymol = pyscf.gto.Mole(atom=molecule, spin=0, charge=0, basis=basis)
pymol.build()
mf = pyscf.scf.RHF(pymol).run()
s = mf.get_ovlp(pymol)

lo = pyimport("pyscf.lo.orth")
lo_ao = lo.lowdin(s)
println("size of Lowdin ortho AO's:", size(lo_ao))


converged SCF energy = -22.7370439057951
size of Lowdin ortho AO's:(40, 40)


In [ ]:
#write fci dump file from the modified mo coefficients
tools = pyimport("pyscf.tools")
tools.fcidump.from_mo(pymol, "fcidump.he08_02", lo_ao)